In [9]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import wandb
from pytorch_lightning.loggers import WandbLogger
from opf.utils import model_from_parameters

%aimport opf.utils
%aimport opf.modules
%aimport opf.powerflow

In [11]:
params = dict(
    case_name="case30",
    adj_scaling="auto",
    adj_threshold=0.01,
    batch_size=256,
    max_epochs=1,
    patience=1000,
    K=8,
    F=32,
    L=2,
    s=10,
    t=500,
    F_MLP=4,
    L_MLP=1,
    cost_weight=0.01,
    lr=1e-4,
    eps=1e-4,
    constraint_features=False,
)

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
log_dir = os.path.join(root_dir, "logs")

logger = WandbLogger(project="opf", save_dir=log_dir, config=params, tags=["test"])
barrier, trainer, dm = model_from_parameters(
    params,
    gpus=0,
    logger=logger,
    data_dir=data_dir,
    patience=params["patience"],
    eps=params["eps"],
    debug=False
)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/damow/.pyenv/versions/3.9.5/envs/opf/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1295: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


In [12]:
trainer.tune(barrier, datamodule=dm)
trainer.fit(barrier, dm)
trainer.test(datamodule=dm)
logger.finalize("finished")
wandb.finish()


  | Name                 | Type                | Params
-------------------------------------------------------------
0 | model                | GNN                 | 124 K 
1 | powerflow_parameters | PowerflowParameters | 0     
-------------------------------------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.993     Total estimated model params size (MB)


AssertionError: 